In [1]:
# Load dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sts
import numpy as np

Notes:
* From looking at the data, the per day draw tons are not accurate or consistent until June 2001
* Due to data limitations, only data after 2005 will be used in regression analysis. The percent drawn is unknown for many drawpoints, therefor the assumption will be made that the cumulative tons from the ORHISTQuery is equal to the total tons for each drawpoint
* Starting in 2005 gives 15 full years of data and should remove, for the most part, any drawpoints with questionable exhaustion

In [2]:
# SQLite dependencies and loads
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy.orm import Session
Base = declarative_base()
# Create sqlite DB file if it does not exist
engine = create_engine("sqlite:///master_db.sqlite")
if not database_exists(engine.url):
    create_database(engine.url)

print(database_exists(engine.url))

True


In [3]:
# Load CSV files into dataframes
ex_data = pd.read_csv('../Data-Analytics-Final_Project/Resources/extraction_data.csv')
exhist_df = pd.read_csv('../Data-Analytics-Final_Project/Resources/exhist_query.csv')

In [4]:
# Adding cumulative tons column for each drawpoint
exhist_df['total_extraction_tons'] = exhist_df.groupby(['extract_point'])['extraction_tons'].apply(lambda x: x.cumsum())

In [5]:
# Sorting by date and drawpoint to verify the cumulative tons works
exhist_df=exhist_df.sort_values(['extract_point','datestring'])
exhist_df.head()

,extract_point,extraction_tons,datestring,total_extraction_tons
583853,611703,405,20151231,405
584233,611703,424,20160105,829
585890,611703,463,20160128,1292
587521,611703,81,20160223,1373
587619,611703,54,20160224,1427


In [6]:
# Find final cumulative tons per drawpoint
totalTons = exhist_df.groupby(['extract_point'])['total_extraction_tons'].max()
totalTons.head()

extract_point
611703    57920
611704    72258
611705    62822
611706    84977
611707    67039
Name: total_extraction_tons, dtype: int64

In [7]:
totalTons_df = pd.DataFrame(totalTons).reset_index()
totalTons_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1281 entries, 0 to 1280
Data columns (total 2 columns):
extract_point            1281 non-null object
total_extraction_tons    1281 non-null int64
dtypes: int64(1), object(1)
memory usage: 20.1+ KB


In [8]:
totalTons_df.to_csv('../Data-Analytics-Final_Project/Resources/totalTons.csv',index=False)

In [9]:
# dp_data is a list of drawpoints, so merging with totalTons_df will remove any non-common drawpoints
total_tons_df = totalTons_df.merge(ex_data,on='extract_point')
total_tons_df.nunique() # confirmed that 1278 drawpoints will be used as they have daily and reserve tons data

extract_point            1278
total_extraction_tons    1264
avg_sA                   1191
reserves                  485
region                      4
dtype: int64

In [10]:
total_tons_df['exhaustion'] = total_tons_df['total_extraction_tons']/total_tons_df['reserves']*100
total_tons_df.head()

,extract_point,total_extraction_tons,avg_sA,reserves,region,exhaustion
0,611703,57920,0.202857,76860,7700SW,75.357793
1,611704,72258,0.188341,109706,7700SW,65.865130
2,611705,62822,0.216857,76860,7700SW,81.735623
3,611706,84977,0.163949,120639,7700SW,70.439079
4,611707,67039,0.174571,76860,7700SW,87.222222


In [11]:
# Merge orhist_df and total_tons_df on drawpoint to get the reserve tons and block
exhist = exhist_df.merge(total_tons_df,how='left',left_on='extract_point',right_on='extract_point')
exhist = exhist[['extract_point','extraction_tons','datestring','total_extraction_tons_x','reserves','region','avg_sA']]
exhist = exhist.rename(columns={'total_extraction_tons_x':'total_extraction_tons'})
exhist['exhaustion'] = exhist['total_extraction_tons']/exhist['reserves']*100
exhist.head(10)

,extract_point,extraction_tons,datestring,total_extraction_tons,reserves,region,avg_sA,exhaustion
0,611703,405,20151231,405,76860.0,7700SW,0.202857,0.526932
1,611703,424,20160105,829,76860.0,7700SW,0.202857,1.078584
2,611703,463,20160128,1292,76860.0,7700SW,0.202857,1.680978
3,611703,81,20160223,1373,76860.0,7700SW,0.202857,1.786365
4,611703,54,20160224,1427,76860.0,7700SW,0.202857,1.856622
5,611703,66,20160225,1493,76860.0,7700SW,0.202857,1.942493
6,611703,50,20160226,1543,76860.0,7700SW,0.202857,2.007546
7,611703,182,20160301,1725,76860.0,7700SW,0.202857,2.244340
8,611703,132,20160302,1857,76860.0,7700SW,0.202857,2.416081
9,611703,75,20160405,1932,76860.0,7700SW,0.202857,2.513661


In [12]:
# Split out drift and bell from drawpoint for grouping
exhist['zone'] = exhist['extract_point'].str[:3]
exhist['sector'] = exhist['extract_point'].str[-3:]
exhist.dropna()
exhist.head()

,extract_point,extraction_tons,datestring,total_extraction_tons,reserves,region,avg_sA,exhaustion,zone,sector
0,611703,405,20151231,405,76860.0,7700SW,0.202857,0.526932,611,703
1,611703,424,20160105,829,76860.0,7700SW,0.202857,1.078584,611,703
2,611703,463,20160128,1292,76860.0,7700SW,0.202857,1.680978,611,703
3,611703,81,20160223,1373,76860.0,7700SW,0.202857,1.786365,611,703
4,611703,54,20160224,1427,76860.0,7700SW,0.202857,1.856622,611,703


In [13]:
len(exhist)

690062

In [14]:
# Total tons per day
exhist_total = exhist_df.groupby(['datestring'])['extraction_tons'].sum()
exhist_total = pd.DataFrame(exhist_total).reset_index().rename(columns={'extraction_tons':'daily_total_extracted'})
exhist_total.head()

,datestring,daily_total_extracted
0,20010602,3050
1,20010604,23217
2,20010605,46802
3,20010606,45967
4,20010607,7310


In [15]:
# merge daily totals to orhist
exhist = exhist.merge(exhist_total,how='left',left_on='datestring',right_on='datestring')
exhist.head(10)

,extract_point,extraction_tons,datestring,total_extraction_tons,reserves,region,avg_sA,exhaustion,zone,sector,daily_total_extracted
0,611703,405,20151231,405,76860.0,7700SW,0.202857,0.526932,611,703,34288
1,611703,424,20160105,829,76860.0,7700SW,0.202857,1.078584,611,703,28457
2,611703,463,20160128,1292,76860.0,7700SW,0.202857,1.680978,611,703,22431
3,611703,81,20160223,1373,76860.0,7700SW,0.202857,1.786365,611,703,20923
4,611703,54,20160224,1427,76860.0,7700SW,0.202857,1.856622,611,703,20941
5,611703,66,20160225,1493,76860.0,7700SW,0.202857,1.942493,611,703,21757
6,611703,50,20160226,1543,76860.0,7700SW,0.202857,2.007546,611,703,19966
7,611703,182,20160301,1725,76860.0,7700SW,0.202857,2.244340,611,703,25951
8,611703,132,20160302,1857,76860.0,7700SW,0.202857,2.416081,611,703,15344
9,611703,75,20160405,1932,76860.0,7700SW,0.202857,2.513661,611,703,14644


In [16]:
exhist = exhist.dropna()
len(exhist)

689982

In [17]:
# create column for daily percent of total tons per drawpoint
exhist['ep_daily_pct'] = exhist['extraction_tons']/exhist['daily_total_extracted']*100
exhist.head()

,extract_point,extraction_tons,datestring,total_extraction_tons,reserves,region,avg_sA,exhaustion,zone,sector,daily_total_extracted,ep_daily_pct
0,611703,405,20151231,405,76860.0,7700SW,0.202857,0.526932,611,703,34288,1.181171
1,611703,424,20160105,829,76860.0,7700SW,0.202857,1.078584,611,703,28457,1.489967
2,611703,463,20160128,1292,76860.0,7700SW,0.202857,1.680978,611,703,22431,2.064108
3,611703,81,20160223,1373,76860.0,7700SW,0.202857,1.786365,611,703,20923,0.387134
4,611703,54,20160224,1427,76860.0,7700SW,0.202857,1.856622,611,703,20941,0.257867


In [18]:
# create drawpoint-datestring as unique values for SQLite DB
exhist['ep_datestring'] = exhist['extract_point']+exhist['datestring'].map(str)
exhist.head()

,extract_point,extraction_tons,datestring,total_extraction_tons,reserves,region,avg_sA,exhaustion,zone,sector,daily_total_extracted,ep_daily_pct,ep_datestring
0,611703,405,20151231,405,76860.0,7700SW,0.202857,0.526932,611,703,34288,1.181171,61170320151231
1,611703,424,20160105,829,76860.0,7700SW,0.202857,1.078584,611,703,28457,1.489967,61170320160105
2,611703,463,20160128,1292,76860.0,7700SW,0.202857,1.680978,611,703,22431,2.064108,61170320160128
3,611703,81,20160223,1373,76860.0,7700SW,0.202857,1.786365,611,703,20923,0.387134,61170320160223
4,611703,54,20160224,1427,76860.0,7700SW,0.202857,1.856622,611,703,20941,0.257867,61170320160224


In [19]:
# Send orhist dataframe to SQLite DB fo easier work and access
class Exhist(Base):
    __tablename__='exhist'
    extract_point = Column(String(10))
    extraction_tons = Column(Integer)
    datestring = Column(Integer)
    total_extraction_tons = Column(Integer)
    reserves = Column(Integer)
    region = Column(String(10))
    avg_sA = Column(Float)
    exhaustion = Column(Float)
    zone = Column(String(3))
    sector = Column(String(3))
    daily_total_extracted = Column(Integer)
    ep_daily_pct = Column(Float)
    ep_datestring = Column(String(15),primary_key=True)

In [20]:
Base.metadata.tables

immutabledict({'exhist': Table('exhist', MetaData(bind=None), Column('extract_point', String(length=10), table=<exhist>), Column('extraction_tons', Integer(), table=<exhist>), Column('datestring', Integer(), table=<exhist>), Column('total_extraction_tons', Integer(), table=<exhist>), Column('reserves', Integer(), table=<exhist>), Column('region', String(length=10), table=<exhist>), Column('avg_sA', Float(), table=<exhist>), Column('exhaustion', Float(), table=<exhist>), Column('zone', String(length=3), table=<exhist>), Column('sector', String(length=3), table=<exhist>), Column('daily_total_extracted', Integer(), table=<exhist>), Column('ep_daily_pct', Float(), table=<exhist>), Column('ep_datestring', String(length=15), table=<exhist>, primary_key=True, nullable=False), schema=None)})

In [21]:
Base.metadata.create_all(engine)

In [22]:
exhist.to_sql('exhist',con=engine,if_exists='replace',index=False)

In [23]:
engine.execute('SELECT * FROM exhist').fetchall()

[('611703', 405, 20151231, 405, 76860.0, '7700SW', 0.20285714300000002, 0.5269320843091335, '611', '703', 34288, 1.18117125524965, '61170320151231'),
 ('611703', 424, 20160105, 829, 76860.0, '7700SW', 0.20285714300000002, 1.078584439240177, '611', '703', 28457, 1.4899673191130478, '61170320160105'),
 ('611703', 463, 20160128, 1292, 76860.0, '7700SW', 0.20285714300000002, 1.6809784022898777, '611', '703', 22431, 2.064107708082564, '61170320160128'),
 ('611703', 81, 20160223, 1373, 76860.0, '7700SW', 0.20285714300000002, 1.7863648191517045, '611', '703', 20923, 0.3871337762271185, '61170320160223'),
 ('611703', 54, 20160224, 1427, 76860.0, '7700SW', 0.20285714300000002, 1.8566224303929222, '611', '703', 20941, 0.25786734157872115, '61170320160224'),
 ('611703', 66, 20160225, 1493, 76860.0, '7700SW', 0.20285714300000002, 1.9424928441321883, '611', '703', 21757, 0.30335064576917775, '61170320160225'),
 ('611703', 50, 20160226, 1543, 76860.0, '7700SW', 0.20285714300000002, 2.007546187874056